# Simple prospective simulation

Prepared by Omar A. Guerrero (oguerrero@turing.ac.uk, @guerrero_oa)

In this tutorial I will demonstrate how to perform a simple prospective analysis by simulating the indicators forward 10 years and measuring the development gaps that would remain open. I will also show how to implement some intuitive visualisations.

## Importing Python's libraries to manipulate and visualise data

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importing PPI's functions

In [3]:
import requests
url = 'https://raw.githubusercontent.com/oguerrer/ppi/main/source_code/ppi.py'
r = requests.get(url)
with open('ppi.py', 'w') as f:
    f.write(r.text)
import ppi

## Load data

Since we are now performing prospective simulations, the initial conditions are the most updated values, i.e. `IF`.

In [31]:
# INDICATORS
df_indis = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_indicators.csv')
N = len(df_indis)
I0 = df_indis.IF.values # initial values
R = df_indis.instrumental # instrumental indicators
qm = df_indis.qm.values # quality of monitoring
rl = df_indis.rl.values # quality of the rule of law
indis_index = dict([(code, i) for i, code in enumerate(df_indis.seriesCode)]) # used to build the network matrix
Imax = df_indis.maxVals.values
Imin = df_indis.minVals.values

# PARAMETERS
df_params = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/parameters.csv')
alphas = df_params.alpha.values
alphas_prime = df_params.alpha_prime.values
betas = df_params.beta.values

# NETWORK
df_net = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_network.csv')
A = np.zeros((N, N)) # adjacency matrix
for index, row in df_net.iterrows():
    i = indis_index[row.origin]
    j = indis_index[row.destination]
    w = row.weight
    A[i,j] = w

# DISBURSEMENT SCHEDULE
df_exp = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_expenditure.csv')
Bs = df_exp.values[:,1::] # disbursement schedule (assumes that the expenditure programmes are properly sorted)

# RELATIONAL TABLE
df_rela = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_relational_table.csv')
B_dict = {}
for index, row in df_rela.iterrows():
    B_dict[indis_index[row.seriesCode]] = [programme for programme in row.values[1::][row.values[1::].astype(str)!='nan']]

## Simulate

In [30]:
T = 30

output = ppi.run_ppi(I0, alphas, alphas_prime, betas, A=A, R=R, qm=qm, rl=rl,
            Imax=Imax, Imin=Imin, Bs=Bs, B_dict=B_dict, T=T)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [32]:
df_params

,alpha,alpha_prime,beta,T,error_alpha,error_beta,GoF_alpha,GoF_beta
0,alpha,alpha_prime,beta,T,error_alpha,error_beta,GoF_alpha,GoF_beta
1,0.0007026226058790586,8.861145096001833e-05,4.134876768431874,69,0.0042895201770180025,0.05210047645482996,0.886624544748952,0.9426894758996871
2,0.016291730092297226,0.006356722788614017,0.024479285618837592,NaN,0.04857246690193226,0.007472748402465444,0.8731889709047655,0.8505450319506911
3,1.446297432106394e-05,5.678341825017476e-06,0.005854489261002293,NaN,8.700235713857578e-06,0.0014508973076415646,0.9729992684742313,0.9709820538471687
4,1.4527105326803212e-08,0.0004460217006854485,0.047712796134339125,NaN,-0.003647995610418945,-0.005547506423796977,0.6962792645453963,0.9918636572450977
...,...,...,...,...,...,...,...,...
68,2.9830933286704836e-05,9.787571450231805e-05,0.06663435029855717,NaN,-0.00010967818250207095,0.003208741343924487,0.9784944740192018,0.9823519226084153
69,0.0016963162690569206,1.1829465893078853e-06,0.07550369232667124,NaN,0.0009446392504857126,-5.780109726960525e-08,0.9638814404226052,0.999999745675172
70,0.0013051266428194411,3.15093167722607e-06,0.10097501790123342,NaN,0.0031828775825624156,0.015042114093675063,0.9465960137153957,0.9793170931211967
71,0.007644604693804947,0.00010887955185435057,0.0648284768622511,NaN,0.02966921082314844,-0.006054842840727659,0.9007829311454182,0.9897533428849224
